In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.datasets import load_sample_images
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import datasets
from matplotlib import pyplot as plt
import cv2
from analyse import Information
from cleaning import Cleaning



class Preprocessing:
    
    def feature_selection(self,df):
        #As we have the latitude and longitude so we don't need a column location seperately in string format
        df=df.drop(['location'], axis=1)
        
        return df
    def feature_slicing(self,df):
        #Use feature slicing for having a column Year help us to compare the accidents yearly
        df.insert(1, column='Year' , value=df['crash_date'].dt.year)
        return df


    
    def feature_engineering(self,df):
        #Here we see the injured or killed people in accidents and we add a new a column
        reaction_columns = ['number_of_persons_injured', 'number_of_persons_killed',
                            'number_of_pedestrians_injured','number_of_pedestrians_killed',
                            'number_of_cyclist_injured','number_of_cyclist_killed',
                            'number_of_motorist_killed','number_of_motorist_injured']


        df['injured_or_killed'] = df[reaction_columns].any(axis=1)

        df['injured_or_killed'] = df['injured_or_killed'].apply(lambda x : "yes" if x else "no")
       
        
        df['injured_or_killed']=pd.get_dummies(df.injured_or_killed, prefix='injured_or_killed', drop_first="True", )
        df=df.rename(columns={"Target1_yes" : "injured_or_killed"})
        
        
        #Number of persons killed       
        target_columns = ['number_of_persons_killed'
                            ,'number_of_pedestrians_killed'
                            ,'number_of_cyclist_killed',
                            'number_of_motorist_killed']
        df['Target1'] = df[target_columns].any(axis=1)

        df['Target1'] = df['Target1'].apply(lambda x : "yes" if x else "no")     
        pd.get_dummies(df.Target1, prefix='Target1')
        df['Target1'] = pd.get_dummies(df.Target1, prefix='Target1', drop_first="True")
        df=df.rename(columns={"Target1_yes" : "Target1"})

        
        
        

        return df
    
    def new_df_normalization(self,df):
         #Here a new dataframe will be created with 8 normalized columns    

        # Normalize total_bedrooms column
        x_array = np.array(df['number_of_persons_injured'])
        normalized_p_injured = preprocessing.normalize([x_array])
        
        df_normalized = pd.DataFrame(normalized_p_injured).T
        df_normalized=df_normalized.rename(columns={0 : 'People_injured'})
        
        
        x_array = np.array(df['number_of_persons_killed'])
        normalized_people_killed = preprocessing.normalize([x_array])
        n=pd.DataFrame(normalized_people_killed).T
        df_normalized.insert(1, 'People_killed', n)
        
        x_array = np.array(df['number_of_pedestrians_injured'])
        normalized_pedestrians_injured = preprocessing.normalize([x_array])
        n=pd.DataFrame(normalized_pedestrians_injured).T
        df_normalized.insert(2, 'Pedestrians_injured', n)
        
                
        x_array = np.array(df['number_of_pedestrians_killed'])
        normalized_pedestrians_killed = preprocessing.normalize([x_array])
        n=pd.DataFrame(normalized_pedestrians_killed).T
        df_normalized.insert(3, 'Pedestrians_killed', n)
        
        x_array = np.array(df['number_of_cyclist_injured'])
        normalized_cyclist_injured = preprocessing.normalize([x_array])
        n=pd.DataFrame(normalized_cyclist_injured).T
        df_normalized.insert(4, 'cyclist_injured', n)
        
        x_array = np.array(df['number_of_cyclist_killed'])
        normalized_cyclist_killed = preprocessing.normalize([x_array])
        n=pd.DataFrame(normalized_cyclist_killed).T
        df_normalized.insert(5, 'cyclist_killed', n)
        
        x_array = np.array(df['number_of_motorist_injured'])
        normalized_motorist_injured = preprocessing.normalize([x_array])
        n=pd.DataFrame(normalized_motorist_injured).T
        df_normalized.insert(6, 'motorist_injured', n)
        
        
        x_array = np.array(df['number_of_motorist_killed'])
        normalized_motorist_killed = preprocessing.normalize([x_array])
        n=pd.DataFrame(normalized_motorist_killed).T
        df_normalized.insert(7, 'motorist_killed', n)
        
        return df_normalized
    
    def resampling(self,df):
        #For a target of killed people we can see that with resampling we saw that 184 people died in accident
        #Now we can see the reasons why they were killed relating to "contributing factor vehicle 1 "


        X = df['contributing_factor_vehicle_1'].to_numpy()
        y=df['Target1'].to_numpy()


         
        class_one_idx = np.argwhere(y==0)
        class_zero_idx = np.argwhere(y==1)

        class_one = np.squeeze(X[class_one_idx])
        class_zero = np.squeeze(X[class_zero_idx])
        print("Number of accidents that cause to kill people : ", class_zero.shape)
        print("Number of accidents that nobody were killed : ", class_one.shape)
        
        
        
        